In [1]:
from IPython.core.debugger import set_trace

#%load_ext nb_black

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import time

plt.style.use(style="dark_background")

In [2]:
SENTIMENT_LABELLED_DATA_FILEPATH = '..\Data\Sentiment Analysis Dataset.csv'
DATA_PREPROCESSING_FILEPATH = '../Data Preprocessed'

TOKENIZER_OUTPUT_FILEPATH = 'tokenizer_100K.pickle'
MODEL_OUTPUT_FILEPATH = 'model_100K_glove'
# MODEL_OUTPUT_FILEPATH = 'model_100K_glove'
MODEL_OUTPUT_FILEPATH = 'model_100K_glove_91'

Import Dataset

In [3]:
txt_data = pd.read_csv(SENTIMENT_LABELLED_DATA_FILEPATH , sep='|', names=['col1'])

#split data into columns with ','
txt_data = txt_data.col1.str.split(',',  3, expand=True)
txt_data.columns = list(txt_data.iloc[0])
txt_data = txt_data.drop(0)
txt_data.index = np.subtract(txt_data.index, 1)
# x = txt_data.groupby('Sentiment')
# l=[x.get_group(i)['SentimentText'] for i in x.groups]


In [4]:
dataset = pd.concat([txt_data['SentimentText'], txt_data['Sentiment']], axis = 1)
dataset.columns = ['text', 'target']

In [5]:

print(dataset.info(verbose=True))
dataset.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1577838 entries, 0 to 1577837
Data columns (total 2 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   text    1577838 non-null  object
 1   target  1577838 non-null  object
dtypes: object(2)
memory usage: 36.1+ MB
None


,text,target
0,is so sad for my APL frie...,0
1,I missed the New Moon trail...,0
2,omg its already 7:30 :O,1
3,.. Omgaga. Im sooo im gunna CRy. I'...,0
4,i think mi bf is cheating on me!!! ...,0


In [6]:
#suffle
dataset = dataset.sample(frac = 1)
dataset

,text,target
501097,@Toxxiccc Thanks Sweetie! It was a good time.,1
1427720,home now harry +.+ i think i could fall in lo...,1
991993,@lisamwiseman LOL I don't have a tripod Gues...,0
256017,"""@iveeanne I wish he did have twitter, but I d...",0
103089,@DaniAlexLuna Got a shout out from @YoungQ NK'...,1
...,...,...
1011082,just got in from school + fkin hell dance wos ...,0
834230,I just broke a nail FML,0
915392,I went salsa dancing tonight w friends. so muc...,0
1459982,I'm sittin in my office Ready for tonite! Gr...,0


In [7]:
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.append(DATA_PREPROCESSING_FILEPATH)
from data_preprocess import *

import pickle

In [9]:
text = NLP_preprocess(dataset)
text.preprocess_data()

text.set_tokenizer()
import pickle

# saving
# with open(TOKENIZER_OUTPUT_FILEPATH, 'wb') as handle:
#     pickle.dump(text.tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [10]:
train_dataset = dataset[:100000]
#train_dataset = dataset[:1000000]
train_labels = train_dataset.target.astype(float)
train_padded = text.tokenize_and_pad(train_dataset, train = True)
print(train_padded)
# saving
# with open(TOKENIZER_OUTPUT_FILEPATH, 'wb') as handle:
#     pickle.dump(text.tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

[[ 27973     30   1184 ...      0      0      0]
 [    31    980     28 ...      0      0      0]
 [ 27975     16      8 ...      0      0      0]
 ...
 [     8     77      4 ...      0      0      0]
 [   205    157    176 ...      0      0      0]
 [ 26896 107248    102 ...      0      0      0]]


In [11]:
text.set_embedding_matrix()

In [12]:
pd.DataFrame(text.embedding_matrix).info


<bound method DataFrame.info of               0         1         2         3        4        5        6   \
0       0.000000  0.000000  0.000000  0.000000  0.00000  0.00000  0.00000   
1       0.000000  0.000000  0.000000  0.000000  0.00000  0.00000  0.00000   
2      -0.673910  0.544500 -0.448680  0.167630 -0.22777  0.56469  0.23045   
3       0.091552  0.093336 -0.028113  0.369900  0.18956  0.43191  0.10266   
4      -0.111640  0.353960 -0.216360 -0.089658  0.42865 -0.58209  0.23430   
...          ...       ...       ...       ...      ...      ...      ...   
107245  0.000000  0.000000  0.000000  0.000000  0.00000  0.00000  0.00000   
107246 -0.737490 -0.406960 -0.035353  0.027531  0.35416 -1.01570  0.19626   
107247  0.000000  0.000000  0.000000  0.000000  0.00000  0.00000  0.00000   
107248  0.000000  0.000000  0.000000  0.000000  0.00000  0.00000  0.00000   
107249  0.000000  0.000000  0.000000  0.000000  0.00000  0.00000  0.00000   

             7         8         9   ...   

In [13]:
test_dataset = dataset[:20000]
test_labels = test_dataset.target.astype(float)
test_padded = text.tokenize_and_pad(test_dataset, train = False)
print(test_padded)

[[27973    30  1184 ...     0     0     0]
 [   31   980    28 ...     0     0     0]
 [27975    16     8 ...     0     0     0]
 ...
 [  337   128    52 ...     0     0     0]
 [43754 43755    93 ...     0     0     0]
 [ 5382  1637   437 ...     0     0     0]]


In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.initializers import Constant
from tensorflow.keras.optimizers import Adam

model = Sequential()

# trainable = False
#bidirectional

#Glove
model.add(
    Embedding(
        text.num_words,
        100,
        embeddings_initializer = Constant(text.embedding_matrix),
        input_length = text.max_length,
        trainable = False,
    )
)

#basic
# for i in range(0,200):
# #model.add(Embedding(text.num_words, 8, input_length=text.max_length))
#     model.add(Dense(1000, activation='relu'))
model.add(LSTM(100, dropout=0.1))
model.add(Dense(1, activation="sigmoid"))

optimizer = Adam(learning_rate=3e-4)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

NotImplementedError: Cannot convert a symbolic Tensor (lstm/strided_slice:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported

In [ ]:
model.summary()

In [ ]:
history = model.fit(
    train_padded, train_labels, epochs=40, validation_data=(test_padded, test_labels),
)

In [22]:
model.save(MODEL_OUTPUT_FILEPATH)

INFO:tensorflow:Assets written to: model_100K_glove_91\assets


INFO:tensorflow:Assets written to: model_100K_glove_91\assets


In [19]:
import random
#test_neg = random.choice(neg.values)
i = random.choice(test_dataset.index)
if test_dataset.loc[i]['target'] == '0':
    print('negative: ', test_dataset.loc[i]['text'])
else:
    print('positive: ', test_dataset.loc[i]['text'])
test_sequences = text.tokenizer.texts_to_sequences([test_dataset.loc[i]['text']])
test_padded2 = pad_sequences(
    test_sequences, maxlen=text.max_length, padding="post", truncating="post"
)
model.predict(test_padded2)

negative:  ohmymandy lucky met


array([[0.8493518]], dtype=float32)

In [31]:
# import keras
# model = keras.models.load_model('C:/Users/Konst/Desktop/PythonNotebooks')

In [ ]:
# model.summary()

In [ ]:
#model.save('')